In [ ]:
# Install all required packages
!pip -q install --upgrade datasets sentence-transformers scikit-learn torch pandas transformers

import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Try to import simalign, fallback to manual if fails
try:
    from simalign import SentenceAligner
    USE_SIMALIGN = True
    print("simalign loaded successfully")
except:
    print("simalign not available, will use manual alignment")
    USE_SIMALIGN = False
    from transformers import AutoTokenizer, AutoModel
    import torch

print("Setup complete!")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.

In [ ]:
!pip -q install datasets sentence-transformers scikit-learn pandas

from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import json
from collections import defaultdict
from tqdm import tqdm

print(" Setup complete!")


✅ Setup complete!


In [ ]:
LANGUAGES = {
    "hi": "Hindi",
    "bn": "Bengali",
    "gu": "Gujarati",
    "mr": "Marathi",
    "pa": "Punjabi",
    "ta": "Tamil",
    "te": "Telugu",
    "ml": "Malayalam"
}

SAMPLE_SIZE = 2000

print("Loading WikiANN datasets...\n")
datasets = {}

for code, name in LANGUAGES.items():
    print(f"Loading {name} ({code})...")
    try:
        ds = load_dataset("wikiann", code)
        full_data = ds["train"]

        if len(full_data) > SAMPLE_SIZE:
            datasets[code] = full_data.select(range(SAMPLE_SIZE))
        else:
            datasets[code] = full_data

        print(f"  {len(datasets[code])} sentences loaded")
    except Exception as e:
        print(f"  ✗ Error: {e}")

print(f"\n Loaded {len(datasets)} languages")
print(f"\n Sample (Hindi): {datasets['hi'][0]['tokens'][:5]}")
print(f"   NER tags: {datasets['hi'][0]['ner_tags'][:5]}")


📥 Loading WikiANN datasets...

Loading Hindi (hi)...


README.md: 0.00B [00:00, ?B/s]

hi/validation-00000-of-00001.parquet:   0%|          | 0.00/64.7k [00:00<?, ?B/s]

hi/test-00000-of-00001.parquet:   0%|          | 0.00/65.0k [00:00<?, ?B/s]

hi/train-00000-of-00001.parquet:   0%|          | 0.00/312k [00:00<?, ?B/s]

...

ml/train-00000-of-00001.parquet:   0%|          | 0.00/988k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

  ✓ 2000 sentences loaded

✅ Loaded 8 languages

📋 Sample (Hindi): ['टैपी', 'ने', 'अपने', 'उत्पादों', 'को']
   NER tags: [0, 0, 0, 0, 0]


In [ ]:
print(f"\n Loaded {len(datasets)} languages")
print(f"\n Sample (Hindi): {datasets['hi'][0]['tokens'][:]}")
print(f"   NER tags: {datasets['hi'][0]['ner_tags'][:1000]}")



✅ Loaded 8 languages

📋 Sample (Hindi): ['टैपी', 'ने', 'अपने', 'उत्पादों', 'को', 'एशिया', 'के', 'अपतटीय', 'भागों', 'में', 'भेजने', 'का', 'फैसला', 'लिया।', 'उन्होंने', 'प्रोमोशन', 'के', 'लिए', 'मैडोना', 'को', 'अपने', 'साथ', 'मिलाया।']
   NER tags: [0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]


In [ ]:
print(" Loading alignment model...")
model = SentenceTransformer('sentence-transformers/LaBSE')
print("  Ready\n")

def find_parallel_pairs(src_data, tgt_data, src_name, tgt_name, threshold=0.75, max_pairs=500):
    print(f"Finding parallels: {src_name} ↔ {tgt_name}")

    src_texts = [" ".join(ex["tokens"]) for ex in src_data]
    tgt_texts = [" ".join(ex["tokens"]) for ex in tgt_data]

    print(f"  Encoding {src_name}...")
    src_embeds = model.encode(src_texts[:1000], batch_size=64, show_progress_bar=False)

    print(f"  Encoding {tgt_name}...")
    tgt_embeds = model.encode(tgt_texts[:1000], batch_size=64, show_progress_bar=False)

    similarities = cosine_similarity(src_embeds, tgt_embeds)

    pairs = []
    for i in range(len(src_embeds)):
        best_idx = int(similarities[i].argmax())  # ← FIXED: Convert to int
        score = float(similarities[i][best_idx])  # ← FIXED: Convert to float

        if score >= threshold and len(pairs) < max_pairs:
            # Access with int indices
            src_ex = src_data[int(i)]
            tgt_ex = tgt_data[best_idx]

            pairs.append({
                "src_idx": int(i),
                "tgt_idx": best_idx,
                "similarity": score,
                "src_tokens": src_ex["tokens"],
                "tgt_tokens": tgt_ex["tokens"],
                "src_ner": src_ex["ner_tags"],
                "tgt_ner": tgt_ex["ner_tags"]
            })

    print(f"  Found {len(pairs)} parallel pairs (≥{threshold})\n")
    return pairs

# Find parallels for all languages
all_parallel = {}
hindi_data = datasets["hi"]

for tgt_code in ["bn", "gu", "mr", "pa", "ta", "te", "ml"]:
    if tgt_code in datasets:
        pairs = find_parallel_pairs(
            hindi_data,
            datasets[tgt_code],
            "Hindi",
            LANGUAGES[tgt_code],
            threshold=0.75,
            max_pairs=300
        )
        all_parallel[tgt_code] = pairs
    else:
        print(f" Skipping {LANGUAGES[tgt_code]} (not loaded)")

print(f"\n Found parallel data for {len(all_parallel)} language pairs!")


🤖 Loading alignment model...
  ✓ Ready

Finding parallels: Hindi ↔ Bengali
  Encoding Hindi...
  Encoding Bengali...
  ✓ Found 75 parallel pairs (≥0.75)

Finding parallels: Hindi ↔ Gujarati
  Encoding Hindi...
  Encoding Gujarati...
  ✓ Found 1 parallel pairs (≥0.75)

Finding parallels: Hindi ↔ Marathi
  Encoding Hindi...
  Encoding Marathi...
  ✓ Found 60 parallel pairs (≥0.75)

Finding parallels: Hindi ↔ Punjabi
  Encoding Hindi...
  Encoding Punjabi...
  ✓ Found 5 parallel pairs (≥0.75)

Finding parallels: Hindi ↔ Tamil
  Encoding Hindi...
  Encoding Tamil...
  ✓ Found 38 parallel pairs (≥0.75)

Finding parallels: Hindi ↔ Telugu
  Encoding Hindi...
  Encoding Telugu...
  ✓ Found 24 parallel pairs (≥0.75)

Finding parallels: Hindi ↔ Malayalam
  Encoding Hindi...
  Encoding Malayalam...
  ✓ Found 34 parallel pairs (≥0.75)


✅ Found parallel data for 7 language pairs!


In [ ]:
print(" Saving parallel corpora...\n")

for lang_code, pairs in all_parallel.items():
    lang_name = LANGUAGES[lang_code]

    # Save to JSON
    filename = f"parallel_hi_{lang_code}_aligned.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(pairs[:100], f, ensure_ascii=False, indent=2)

    print(f"  Saved {filename} ({len(pairs)} pairs)")

# Create summary
summary = {
    "method": "Word alignment using LaBSE embeddings + cosine similarity",
    "source_language": "Hindi",
    "target_languages": [LANGUAGES[code] for code in all_parallel.keys()],
    "total_pairs": sum(len(pairs) for pairs in all_parallel.values()),
    "similarity_threshold": 0.75,
    "results": {
        LANGUAGES[code]: {
            "pairs": len(pairs),
            "avg_similarity": float(np.mean([p["similarity"] for p in pairs]))
        }
        for code, pairs in all_parallel.items()
    }
}

with open("alignment_summary.json", "w") as f:
    json.dump(summary, f, indent=2)

print(f"\n All files saved!")


Samantar


In [ ]:
!pip install transformers sentencepiece torch simalign indic-nlp-library


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 8.3 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

# English-Hindi and English-Tamil pairs
hi_ds = load_dataset("ai4bharat/samanantar", "hi", split="train[:2%]")  # just 2% for quick demo
ta_ds = load_dataset("ai4bharat/samanantar", "ta", split="train[:2%]")
print(hi_ds[0])
print(ta_ds[0])


README.md: 0.00B [00:00, ?B/s]

hi/train-00000-of-00008.parquet:   0%|          | 0.00/240M [00:00<?, ?B/s]

hi/train-00001-of-00008.parquet:   0%|          | 0.00/240M [00:00<?, ?B/s]

hi/train-00002-of-00008.parquet:   0%|          | 0.00/240M [00:00<?, ?B/s]

hi/train-00003-of-00008.parquet:   0%|          | 0.00/240M [00:00<?, ?B/s]

...

ta/train-00001-of-00004.parquet:   0%|          | 0.00/189M [00:00<?, ?B/s]

ta/train-00002-of-00004.parquet:   0%|          | 0.00/188M [00:00<?, ?B/s]

ta/train-00003-of-00004.parquet:   0%|          | 0.00/188M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5264867 [00:00<?, ? examples/s]

{'idx': 0, 'src': "However, Paes, who was partnering Australia's Paul Hanley, could only go as far as the quarterfinals where they lost to Bhupathi and Knowles", 'tgt': 'आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाले पेस मियामी में क्वार्टरफाइनल तक ही पहुंच सके क्योंकि इस दौर में उन्हें भूपति और नोल्स ने हराया था।'}
{'idx': 0, 'src': 'Some 14 months later, the second calf is born.', 'tgt': 'சுமார் 14 மாதங்கள் கழித்து, இரண்டாம் கன்றை ஈனுகிறது.'}


In [ ]:
from datasets import load_dataset

hi_ds = load_dataset("ai4bharat/samanantar", "hi", split="train[:1%]")
print(hi_ds[0])


{'idx': 0, 'src': "However, Paes, who was partnering Australia's Paul Hanley, could only go as far as the quarterfinals where they lost to Bhupathi and Knowles", 'tgt': 'आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाले पेस मियामी में क्वार्टरफाइनल तक ही पहुंच सके क्योंकि इस दौर में उन्हें भूपति और नोल्स ने हराया था।'}


In [ ]:
from datasets import load_dataset

# Load English–Hindi and English–Tamil subsets
hi_ds = load_dataset("ai4bharat/samanantar", "hi", split="train[:2%]")
ta_ds = load_dataset("ai4bharat/samanantar", "ta", split="train[:2%]")

# Build English → Tamil map
ta_map = {ex["src"]: ex["tgt"] for ex in ta_ds}

pairs = []
for ex in hi_ds:
    eng = ex["src"]
    if eng in ta_map:
        pairs.append((ex["tgt"], ta_map[eng]))  # (Hindi, Tamil)

print(" Found", len(pairs), "Hindi–Tamil pairs")
if pairs:
    print("Example:\nHindi:", pairs[0][0], "\nTamil:", pairs[0][1])


✅ Found 4029 Hindi–Tamil pairs
Example:
Hindi: इस पर कार्यकर्ताओं पर कोई कार्रवाई नहीं की गई है। 
Tamil: இதுகுறித்து புகாா் அளித்தும் நடவடிக்கை எடுக்கப்படுவதில்லை.


In [ ]:
!pip install transformers simalign torch --quiet

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

model_name = "mirfan899/hindi-bert-ner"  # public Hindi NER model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Use a pipeline for easy tagging
ner_pipe = pipeline(
    "ner",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple"
)

# Quick test
sample_hi = pairs[0][0]
print("Hindi sample:\n", sample_hi)
print("\nNER output:\n", ner_pipe(sample_hi))

tokenizer_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/709M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

Device set to use cpu


Hindi sample:
 इस पर कार्यकर्ताओं पर कोई कार्रवाई नहीं की गई है।

NER output:
 []


In [ ]:
from simalign import SentenceAligner

# Initialize aligner (uses multilingual XLM-R model)
# aligner = SentenceAligner(model="xlmr", token_type="bpe", matching_methods=["itermax"])
aligner = SentenceAligner(model="xlmr")



# Test alignment on first example
src = pairs[0][0].split()  # Hindi tokens
tgt = pairs[0][1].split()  # Tamil tokens

alignment = aligner.get_word_aligns(src, tgt)
print("\nSample alignment pairs (hi_index, ta_index):")
print(alignment["itermax"][:10])


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

2025-11-06 17:19:42,953 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: xlm-roberta-base
INFO:simalign.simalign:Initialized the EmbeddingLoader with model: xlm-roberta-base



Sample alignment pairs (hi_index, ta_index):
[(0, 0), (1, 0), (5, 3), (6, 4), (7, 4), (8, 4), (9, 0), (9, 4)]


In [ ]:
projected = []

# Limit for demo
num_samples = min(50, len(pairs))

for i in range(num_samples):
    hi_sent, ta_sent = pairs[i]

    # Step 1: Run NER on Hindi
    hi_ner = ner_pipe(hi_sent)
    hi_words = hi_sent.split()
    ta_words = ta_sent.split()

    # Step 2: Align
    align_data = aligner.get_word_aligns(hi_words, ta_words)
    aligns = align_data["itermax"]

    # Step 3: Build Hindi token → NER label map
    hi_map = {}
    for ent in hi_ner:
        for token in ent["word"].split():
            try:
                idx = hi_words.index(token)
                hi_map[idx] = ent["entity_group"]
            except ValueError:
                continue

    # Step 4: Project tags to Tamil side
    ta_tags = ["O"] * len(ta_words)
    for (hi_idx, ta_idx) in aligns:
        if hi_idx in hi_map:
            ta_tags[ta_idx] = hi_map[hi_idx]

    projected.append({
        "hindi": hi_sent,
        "tamil": ta_sent,
        "tamil_tags": ta_tags
    })

In [ ]:
import json

out_file = "projected_hi_ta_ner.jsonl"
with open(out_file, "w", encoding="utf-8") as f:
    for p in projected:
        f.write(json.dumps(p, ensure_ascii=False) + "\n")

print(f"\n Saved {len(projected)} projected Hindi–Tamil NER-tagged pairs to {out_file}")




✅ Saved 50 projected Hindi–Tamil NER-tagged pairs to projected_hi_ta_ner.jsonl


In [ ]:
print("\nExample projection:\n")
for w, tag in zip(projected[0]["tamil"].split(), projected[0]["tamil_tags"]):
    print(f"{w}\t{tag}")


Example projection:

இதுகுறித்து	O
புகாா்	O
அளித்தும்	O
நடவடிக்கை	O
எடுக்கப்படுவதில்லை.	O


In [ ]:
test_hi = "नरेंद्र मोदी दिल्ली में हैं।"
test_ta = "நரேந்திர மோடி டெல்லியில் உள்ளார்."
print(ner_pipe(test_hi))
src = test_hi.split()
tgt = test_ta.split()
alignment = aligner.get_word_aligns(src, tgt)
# projection logic same as before


[{'entity_group': 'LOCATION', 'score': np.float32(0.93002474), 'word': 'नरेंद्र मोदी दिल्ली', 'start': 0, 'end': 19}]


way 2


In [ ]:

from datasets import load_dataset

# Load small subset for demo (~2% of data)
hi_ds = load_dataset("ai4bharat/samanantar", "hi", split="train[:2%]")
ta_ds = load_dataset("ai4bharat/samanantar", "ta", split="train[:2%]")

# Build English → Tamil map
ta_map = {ex["src"]: ex["tgt"] for ex in ta_ds}

# Construct Hindi–Tamil pairs by pivoting through English
pairs = []
for ex in hi_ds:
    eng = ex["src"]
    if eng in ta_map:
        pairs.append((ex["tgt"], ta_map[eng]))  # (Hindi, Tamil)

print(f" Found {len(pairs)} Hindi–Tamil pairs")
if pairs:
    print("Example:\nHindi:", pairs[0][0], "\nTamil:", pairs[0][1])

from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# Multilingual high-resource NER model (handles Hindi well)
model_name = "Davlan/xlm-roberta-base-ner-hrl"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Create NER pipeline
ner_pipe = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# Quick test on Hindi text
sample_hi = "नरेंद्र मोदी दिल्ली में हैं।"
print("\nHindi sample:", sample_hi)
print("NER output:", ner_pipe(sample_hi))

from simalign import SentenceAligner

# Initialize aligner (default settings, XLM-R)
aligner = SentenceAligner(model="xlmr")

# Test alignment for one sentence pair
src = pairs[0][0].split()
tgt = pairs[0][1].split()

alignment = aligner.get_word_aligns(src, tgt)
print("\nSample alignment pairs (hi_index, ta_index):")
print(alignment["itermax"][:10])

projected = []

num_samples = min(50, len(pairs))  # process limited pairs for demo

for i in range(num_samples):
    hi_sent, ta_sent = pairs[i]

    # Step 1: Run NER on Hindi sentence
    hi_ner = ner_pipe(hi_sent)
    hi_words = hi_sent.split()
    ta_words = ta_sent.split()

    # Step 2: Align tokens
    align_data = aligner.get_word_aligns(hi_words, ta_words)
    aligns = align_data["itermax"]

    # Step 3: Map Hindi token index → NER label
    hi_map = {}
    for ent in hi_ner:
        for token in ent["word"].split():
            try:
                idx = hi_words.index(token)
                hi_map[idx] = ent["entity_group"]
            except ValueError:
                continue

    # Step 4: Project tags to Tamil side
    ta_tags = ["O"] * len(ta_words)
    for (hi_idx, ta_idx) in aligns:
        if hi_idx in hi_map:
            ta_tags[ta_idx] = hi_map[hi_idx]

    projected.append({
        "hindi": hi_sent,
        "tamil": ta_sent,
        "tamil_tags": ta_tags
    })

import json

out_file = "projected_hi_ta_ner.jsonl"
with open(out_file, "w", encoding="utf-8") as f:
    for p in projected:
        f.write(json.dumps(p, ensure_ascii=False) + "\n")

print(f"\n✅ Done. Saved {len(projected)} Hindi–Tamil NER projected pairs → {out_file}")

print("\nExample Tamil Projection:\n")
for w, tag in zip(projected[0]["tamil"].split(), projected[0]["tamil_tags"]):
    print(f"{w}\t{tag}")


✅ Found 4029 Hindi–Tamil pairs
Example:
Hindi: इस पर कार्यकर्ताओं पर कोई कार्रवाई नहीं की गई है। 
Tamil: இதுகுறித்து புகாா் அளித்தும் நடவடிக்கை எடுக்கப்படுவதில்லை.


tokenizer_config.json:   0%|          | 0.00/211 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/980 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Device set to use cpu



Hindi sample: नरेंद्र मोदी दिल्ली में हैं।
NER output: [{'entity_group': 'PER', 'score': np.float32(0.99941605), 'word': 'नरेंद्र मोदी', 'start': 0, 'end': 12}, {'entity_group': 'LOC', 'score': np.float32(0.9994332), 'word': 'दिल्ली', 'start': 12, 'end': 19}]


2025-11-06 17:26:29,165 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: xlm-roberta-base
INFO:simalign.simalign:Initialized the EmbeddingLoader with model: xlm-roberta-base



Sample alignment pairs (hi_index, ta_index):
[(0, 0), (1, 0), (5, 3), (6, 4), (7, 4), (8, 4), (9, 0), (9, 4)]

✅ Done. Saved 50 Hindi–Tamil NER projected pairs → projected_hi_ta_ner.jsonl

Example Tamil Projection:

இதுகுறித்து	O
புகாா்	O
அளித்தும்	O
நடவடிக்கை	O
எடுக்கப்படுவதில்லை.	O


In [ ]:
import json

def project_single_pair(hindi_sent, target_sent, ner_pipe, aligner):
    # 1) Get NER results (character spans or aggregated strings)
    ner_out = ner_pipe(hindi_sent)  # list of dicts with 'word', 'entity_group', maybe 'start','end'
    # Normalize ner spans to list of (start,end,label,text)
    spans = []
    for ent in ner_out:
        label = ent.get("entity_group") or ent.get("entity")
        # prefer char spans if present
        if "start" in ent and "end" in ent:
            spans.append((ent["start"], ent["end"], label, ent.get("word")))
        else:
            # fallback: find substring occurrence(s) of ent["word"] in the sentence
            text = ent["word"].strip()
            if not text:
                continue
            # find all occurrences and take first (simple fallback)
            idx = hindi_sent.find(text)
            if idx != -1:
                spans.append((idx, idx + len(text), label, text))
    # If no spans, nothing to project
    if not spans:
        return {"target_tokens": target_sent.split(), "target_tags": ["O"] * len(target_sent.split()), "ner_spans": spans}

    # 2) Ask SimAlign for alignment (pass raw strings so it will tokenize consistently)
    align_data = aligner.get_word_aligns(hindi_sent, target_sent)
    # print(align_data.keys())  # use this if you want to inspect keys

    # find source tokenization used by SimAlign (try common keys)
    src_token_keys = [k for k in ("src_tokens","src_words","source_tokens","s_tokens","src","source") if k in align_data]
    trg_token_keys = [k for k in ("trg_tokens","trg_words","target_tokens","t_tokens","trg","target") if k in align_data]

    if not src_token_keys or not trg_token_keys:
        # fallback to whitespace tokenization (less accurate)
        src_tokens = hindi_sent.split()
        trg_tokens = target_sent.split()
        aligns = align_data.get("itermax", [])
    else:
        src_tokens = align_data[src_token_keys[0]]
        trg_tokens = align_data[trg_token_keys[0]]
        aligns = align_data.get("itermax", [])

    # 3) Build char-offset mapping for the source tokens (join tokens with a single separator and map)
    # We need a mapping from src_token_index -> (char_start, char_end) in the original hindi_sent.
    # Approach: iterate tokens and find them in the sentence greedily.
    token_char_spans = []
    cursor = 0
    for tok in src_tokens:
        tok_stripped = tok
        # if tokens contain special markers (like '▁' or '##'), normalize
        # remove leading underscore used by some tokenizers (e.g., sentencepiece uses '▁')
        tok_norm = tok_stripped.replace("▁", "")
        # find the next occurrence of tok_norm at or after cursor
        idx = hindi_sent.find(tok_norm, cursor)
        if idx == -1:
            # fallback: try to skip whitespace and continue
            idx = hindi_sent.find(tok_norm)
        if idx == -1:
            # last resort: treat as zero-length
            token_char_spans.append((cursor, cursor, tok))
            # don't move cursor
        else:
            token_char_spans.append((idx, idx + len(tok_norm), tok))
            cursor = idx + len(tok_norm)

    # 4) For each NER span, find which src token indices overlap it
    src_token_to_label = {}  # map token index -> label
    for (sstart, send, label, text) in spans:
        for i, (tstart, tend, tok) in enumerate(token_char_spans):
            # consider overlap
            if not (tend <= sstart or tstart >= send):
                src_token_to_label[i] = label

    # 5) Project labels through alignments
    trg_tags = ["O"] * len(trg_tokens)
    for (si, ti) in aligns:
        if si in src_token_to_label:
            trg_tags[ti] = src_token_to_label[si]

    return {
        "src_tokens": src_tokens,
        "trg_tokens": trg_tokens,
        "aligns": aligns,
        "ner_spans": spans,
        "target_tags": trg_tags
    }

# Example usage (replace with your variables):
hi_sent = "नरेंद्र मोदी दिल्ली में हैं।"
ta_sent = "நரேந்திர மோடி டெல்லியில் உள்ளார்."
res = project_single_pair(hi_sent, ta_sent, ner_pipe, aligner)
print("SRC TOKENS:", res["src_tokens"])
print("TRG TOKENS:", res["trg_tokens"])
print("ALIGNMENTS:", res["aligns"])
print("NER SPANS:", res["ner_spans"])
print("PROJECTED TAGS:", list(zip(res["trg_tokens"], res["target_tags"])))


SRC TOKENS: ['नरेंद्र', 'मोदी', 'दिल्ली', 'में', 'हैं।']
TRG TOKENS: ['நரேந்திர', 'மோடி', 'டெல்லியில்', 'உள்ளார்.']
ALIGNMENTS: [(0, 0), (1, 1), (2, 2), (3, 2), (4, 3)]
NER SPANS: [(0, 12, 'PER', 'नरेंद्र मोदी'), (12, 19, 'LOC', 'दिल्ली')]
PROJECTED TAGS: [('நரேந்திர', 'PER'), ('மோடி', 'PER'), ('டெல்லியில்', 'LOC'), ('உள்ளார்.', 'O')]


In [ ]:
!pip install simalign

Full pipeline for trnslation

In [ ]:
!pip install simalign

In [3]:
!pip install simalign
import json
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from simalign import SentenceAligner

model_name = "Davlan/xlm-roberta-base-ner-hrl"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

ner_pipe = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

aligner = SentenceAligner(model="xlmr")  # defaults to XLM-R embeddings

def project_ner_pair(hi_sent, tgt_sent):
    """
    Projects NER tags from a Hindi sentence to its parallel target-language sentence.
    Returns dict with target tokens + tags.
    """

    # Step 1: NER tagging for Hindi
    hi_ner = ner_pipe(hi_sent)
    if not hi_ner:
        return None

    # Step 2: Get alignments
    align_data = aligner.get_word_aligns(hi_sent, tgt_sent)
    src_tokens = align_data.get("src", hi_sent.split())
    tgt_tokens = align_data.get("trg", tgt_sent.split())
    aligns = align_data.get("itermax", [])

    # Step 3: Build token span map for Hindi
    token_spans = []
    cursor = 0
    for tok in src_tokens:
        tok_norm = tok.replace("▁", "")
        idx = hi_sent.find(tok_norm, cursor)
        if idx == -1:
            token_spans.append((cursor, cursor))
        else:
            token_spans.append((idx, idx + len(tok_norm)))
            cursor = idx + len(tok_norm)

    # Step 4: Convert NER spans to token labels
    token_labels = ["O"] * len(src_tokens)
    for ent in hi_ner:
        label = ent["entity_group"]
        s, e = ent["start"], ent["end"]
        for i, (ts, te) in enumerate(token_spans):
            if not (te <= s or ts >= e):
                token_labels[i] = label

    # Step 5: Project tags to target language
    tgt_labels = ["O"] * len(tgt_tokens)
    for (si, ti) in aligns:
        if si < len(token_labels):
            if token_labels[si] != "O":
                tgt_labels[ti] = token_labels[si]

    return {
        "hindi": hi_sent,
        "target": tgt_sent,
        "target_tokens": tgt_tokens,
        "target_tags": tgt_labels,
    }


target_langs = ["bn", "gu", "mr", "pa", "ta", "te", "ml", "as"]

for lang in target_langs:
    print(f"\n🌐 Processing Hindi → {lang.upper()} ...")

    # Load Samanantar parallel corpora
    hi_ds = load_dataset("ai4bharat/samanantar", "hi", split="train[:2%]")
    tgt_ds = load_dataset("ai4bharat/samanantar", lang, split="train[:2%]")

    # Build English→Target mapping (since both are aligned to English)
    tgt_map = {ex["src"]: ex["tgt"] for ex in tgt_ds}

    # Build Hindi–Target pairs using English as pivot
    pairs = []
    for ex in hi_ds:
        eng = ex["src"]
        if eng in tgt_map:
            pairs.append((ex["tgt"], tgt_map[eng]))

    # print(f"  Found {len(pairs)} parallel pairs.")

    # Project NER tags for first 100 pairs for demo
    projected = []
    for i, (hi_sent, tgt_sent) in enumerate(pairs[:100]):
        try:
            out = project_ner_pair(hi_sent, tgt_sent)
            if out:
                projected.append(out)
        except Exception as e:
            continue

    # Save to JSONL file
    out_file = f"projected_hi_{lang}_ner.jsonl"
    with open(out_file, "w", encoding="utf-8") as f:
        for p in projected:
            f.write(json.dumps(p, ensure_ascii=False) + "\n")

    # print(f"  ✅ Saved {len(projected)} projected pairs → {out_file}")

if projected:
    print("\nExample projection (last processed language):\n")
    for w, tag in zip(projected[0]["target_tokens"], projected[0]["target_tags"]):
        print(f"{w}\t{tag}")


tokenizer_config.json:   0%|          | 0.00/211 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/980 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

...

te/train-00000-of-00003.parquet:   0%|          | 0.00/200M [00:00<?, ?B/s]

te/train-00001-of-00003.parquet:   0%|          | 0.00/200M [00:00<?, ?B/s]

te/train-00002-of-00003.parquet:   0%|          | 0.00/200M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4946035 [00:00<?, ? examples/s]

KeyboardInterrupt: 

**Evaluation of Dataset**


In [4]:
from transformers import pipeline
import json
from tqdm import tqdm
import os

# Load model once
print("Loading XLM-RoBERTa NER model...")
hindi_ner = pipeline(
    "ner",
    model="Davlan/xlm-roberta-base-wikiann-ner",
    aggregation_strategy="simple",
    device=-1
)
print("✓ Model loaded!\n")

# All language files
language_files = [
    "projected_hi_as_ner.jsonl",
    "projected_hi_bn_ner.jsonl",
    "projected_hi_gu_ner.jsonl",
    "projected_hi_ml_ner.jsonl",
    "projected_hi_mr_ner.jsonl",
    "projected_hi_pa_ner.jsonl",
    "projected_hi_ta_ner.jsonl",
    "projected_hi_te_ner.jsonl"
]

def get_token_tags(text, ner_predictions):
    """Convert entity predictions to token-level tags."""
    tokens = text.split()
    tags = ['O'] * len(tokens)

    char_positions = []
    char_idx = 0
    for token in tokens:
        start = text.find(token, char_idx)
        if start != -1:
            end = start + len(token)
            char_positions.append((start, end))
            char_idx = end
        else:
            char_positions.append((None, None))

    for pred in ner_predictions:
        pred_start = pred['start']
        pred_end = pred['end']
        entity_type = pred['entity_group']

        for i, (tok_start, tok_end) in enumerate(char_positions):
            if tok_start is None:
                continue
            if tok_start < pred_end and tok_end > pred_start:
                tags[i] = entity_type

    return tags

# Process each language
print("="*70)
print("Processing all language files...")
print("="*70 + "\n")

results_summary = []

for filename in language_files:
    if not os.path.exists(filename):
        print(f"Skipping {filename} (not found)")
        continue

    lang_code = filename.split("_")[2]  # Extract 'bn', 'gu', etc.

    print(f"Processing {filename}...")

    # Load data
    with open(filename, 'r', encoding='utf-8') as f:
        data = [json.loads(line) for line in f]

    # Generate Hindi tags
    for item in tqdm(data, desc=f"  {lang_code.upper()}", leave=False):
        try:
            hindi_preds = hindi_ner(item['hindi'])
            hindi_tags = get_token_tags(item['hindi'], hindi_preds)
            item['hindi_tags'] = hindi_tags
        except:
            item['hindi_tags'] = ['O'] * len(item['hindi'].split())

    # Save
    output_file = filename.replace('.json', '_with_hindi_tags.json')
    with open(output_file, 'w', encoding='utf-8') as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

    # Sample check
    sample = data[0]
    hindi_entities = sum(1 for t in sample['hindi_tags'] if t != 'O')
    target_entities = sum(1 for t in sample['target_tags'] if t != 'O')

    results_summary.append({
        'language': lang_code,
        'file': output_file,
        'sentences': len(data),
        'sample_hindi_entities': hindi_entities,
        'sample_target_entities': target_entities
    })

    print(f"  ✓ Saved: {output_file}")
    # print(f"  ✓ Sentences: {len(data)}")
    # print(f"  ✓ Sample - Hindi entities: {hindi_entities}, Target entities: {target_entities}\n")



Loading XLM-RoBERTa NER model...


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

...

  ✓ Saved: projected_hi_ml_ner_with_hindi_tags.jsonl
Processing projected_hi_mr_ner.jsonl...


  ✓ Saved: projected_hi_mr_ner_with_hindi_tags.jsonl
Processing projected_hi_pa_ner.jsonl...


  ✓ Saved: projected_hi_pa_ner_with_hindi_tags.jsonl
Processing projected_hi_ta_ner.jsonl...


  ✓ Saved: projected_hi_ta_ner_with_hindi_tags.jsonl
Processing projected_hi_te_ner.jsonl...


  ✓ Saved: projected_hi_te_ner_with_hindi_tags.jsonl


In [ ]:
import json
from collections import Counter
import numpy as np

# Language files (excluding PA)
language_files = [
    ("projected_hi_as_ner_with_hindi_tags.jsonl", "Assamese"),
    ("projected_hi_bn_ner_with_hindi_tags.jsonl", "Bengali"),
    ("projected_hi_gu_ner_with_hindi_tags.jsonl", "Gujarati"),
    ("projected_hi_ml_ner_with_hindi_tags.jsonl", "Malayalam"),
    ("projected_hi_mr_ner_with_hindi_tags.jsonl", "Marathi"),
    ("projected_hi_ta_ner_with_hindi_tags.jsonl", "Tamil"),
    ("projected_hi_te_ner_with_hindi_tags.jsonl", "Telugu")
]

def calculate_entity_matching(data):
    """
    Calculate how well entities transferred from Hindi to target.

    Returns:
    - entity_count_ratio: ratio of target entities to hindi entities
    - entity_type_precision: % of target entities that have correct type
    """
    total_hindi_entities = 0
    total_target_entities = 0
    matching_entity_count = 0

    for item in data:
        # Count entities in Hindi
        hindi_entities = sum(1 for tag in item['hindi_tags'] if tag != 'O')
        target_entities = sum(1 for tag in item['target_tags'] if tag != 'O')

        total_hindi_entities += hindi_entities
        total_target_entities += target_entities

        # Count how many sentences have similar entity count
        if hindi_entities > 0:
            ratio = target_entities / hindi_entities
            # Consider "matched" if within 20% of each other
            if 0.8 <= ratio <= 1.2:
                matching_entity_count += 1

    # Entity count ratio (should be close to 1.0)
    entity_ratio = total_target_entities / total_hindi_entities if total_hindi_entities > 0 else 0

    # Sentence-level matching rate
    sentence_match_rate = matching_entity_count / len(data) if len(data) > 0 else 0

    return {
        'hindi_entities': total_hindi_entities,
        'target_entities': total_target_entities,
        'entity_ratio': entity_ratio,
        'sentence_match_rate': sentence_match_rate
    }

# Evaluate all languages
print("="*70)
print(" METHOD 1: ENTITY MATCHING ACCURACY")
print("="*70)
print("\nMeasures: How many entities transferred from Hindi to target\n")

all_results = {}

for filename, lang_name in language_files:
    print(f"\n{lang_name}:")
    print("-" * 50)

    # Load data
    with open(filename, 'r', encoding='utf-8') as f:
        data = [json.loads(line) for line in f]

    # Calculate metrics
    metrics = calculate_entity_matching(data)

    print(f"  Entity ratio:      {metrics['entity_ratio']:.3f} (ideal: 1.0)")



    all_results[lang_name] = metrics

# Overall summary
print("\n" + "="*70)
print("📈 OVERALL SUMMARY - METHOD 1")
print("="*70)

avg_ratio = np.mean([r['entity_ratio'] for r in all_results.values()])


print(f"\nAverage entity ratio: {avg_ratio:.3f}")





In [ ]:
import json
from collections import Counter
import numpy as np

language_files = [
    ("projected_hi_as_ner_with_hindi_tags.jsonl", "Assamese"),
    ("projected_hi_bn_ner_with_hindi_tags.jsonl", "Bengali"),
    ("projected_hi_gu_ner_with_hindi_tags.jsonl", "Gujarati"),
    ("projected_hi_ml_ner_with_hindi_tags.jsonl", "Malayalam"),
    ("projected_hi_mr_ner_with_hindi_tags.jsonl", "Marathi"),
    ("projected_hi_ta_ner_with_hindi_tags.jsonl", "Tamil"),
    ("projected_hi_te_ner_with_hindi_tags.jsonl", "Telugu")
]

def calculate_tag_consistency(data):
    """
    Compare entity type distribution between Hindi and target.
    Returns similarity score (0-1, higher is better).
    """
    # Count entity types
    hindi_dist = Counter()
    target_dist = Counter()

    for item in data:
        for tag in item['hindi_tags']:
            if tag != 'O':
                hindi_dist[tag] += 1

        for tag in item['target_tags']:
            if tag != 'O':
                target_dist[tag] += 1

    # Calculate distribution similarity
    all_tags = set(hindi_dist.keys()) | set(target_dist.keys())

    if not all_tags:
        return {
            'hindi_dist': {},
            'target_dist': {},
            'similarity': 0.0,
            'per_type_diff': {}
        }

    # Normalize to percentages
    hindi_total = sum(hindi_dist.values())
    target_total = sum(target_dist.values())

    per_type_diff = {}
    similarity_scores = []

    for tag in all_tags:
        hindi_pct = (hindi_dist.get(tag, 0) / hindi_total * 100) if hindi_total > 0 else 0
        target_pct = (target_dist.get(tag, 0) / target_total * 100) if target_total > 0 else 0

        # Calculate absolute difference
        diff = abs(hindi_pct - target_pct)
        per_type_diff[tag] = diff

        # Similarity for this tag (1 - normalized difference)
        similarity_scores.append(1 - (diff / 100))

    overall_similarity = np.mean(similarity_scores)

    return {
        'hindi_dist': dict(hindi_dist),
        'target_dist': dict(target_dist),
        'similarity': overall_similarity,
        'per_type_diff': per_type_diff
    }

# Evaluate all languages
print("="*70)
print(" METHOD 2: TAG TYPE CONSISTENCY")
print("="*70)
print("\nMeasures: Are entity types (PER/LOC/ORG) distributed similarly?\n")

all_results = {}

for filename, lang_name in language_files:
    print(f"\n{lang_name}:")
    print("-" * 50)

    # Load data
    with open(filename, 'r', encoding='utf-8') as f:
        data = [json.loads(line) for line in f]

    # Calculate metrics
    metrics = calculate_tag_consistency(data)

    # Display distribution
    hindi_total = sum(metrics['hindi_dist'].values())
    target_total = sum(metrics['target_dist'].values())

    print("  Entity Type Distribution:")
    print(f"  {'Type':<8} {'Hindi':<15} {'Target':<15} {'Diff'}")

    for tag in ['PER', 'LOC', 'ORG']:
        h_count = metrics['hindi_dist'].get(tag, 0)
        t_count = metrics['target_dist'].get(tag, 0)
        h_pct = (h_count / hindi_total * 100) if hindi_total > 0 else 0
        t_pct = (t_count / target_total * 100) if target_total > 0 else 0
        diff = metrics['per_type_diff'].get(tag, 0)

        print(f"  {tag:<8}  ({h_pct:>5.1f}%)    ({t_pct:>5.1f}%)   {diff:>5.1f}%")

    print(f"\n  Overall Similarity: {metrics['similarity']:.3f}")



    all_results[lang_name] = metrics

# Overall summary
print("\n" + "="*70)
print("📈 OVERALL SUMMARY - METHOD 2")
print("="*70)

avg_similarity = np.mean([r['similarity'] for r in all_results.values()])

print(f"\nAverage tag consistency: {avg_similarity:.3f}")



In [ ]:
import json
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, classification_report

language_files = [
    ("projected_hi_as_ner_with_hindi_tags.jsonl", "Assamese"),
    ("projected_hi_bn_ner_with_hindi_tags.jsonl", "Bengali"),
    ("projected_hi_gu_ner_with_hindi_tags.jsonl", "Gujarati"),
    ("projected_hi_ml_ner_with_hindi_tags.jsonl", "Malayalam"),
    ("projected_hi_mr_ner_with_hindi_tags.jsonl", "Marathi"),
    ("projected_hi_ta_ner_with_hindi_tags.jsonl", "Tamil"),
    ("projected_hi_te_ner_with_hindi_tags.jsonl", "Telugu")
]

def calculate_alignment_accuracy(data):
    """
    Calculate token-level precision, recall, F1.
    Assumes target tags should match Hindi tags if alignment is perfect.
    """
    all_hindi_tags = []
    all_target_tags = []

    for item in data:
        # Align to shorter length (simple approach)
        min_len = min(len(item['hindi_tags']), len(item['target_tags']))

        all_hindi_tags.extend(item['hindi_tags'][:min_len])
        all_target_tags.extend(item['target_tags'][:min_len])

    # Calculate metrics (treating Hindi as "gold standard")
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_hindi_tags,  # Gold standard
        all_target_tags,  # Predicted
        average='weighted',
        zero_division=0,
        labels=['O', 'PER', 'LOC', 'ORG']
    )

    # Entity-level F1 (excluding 'O')
    entity_precision, entity_recall, entity_f1, _ = precision_recall_fscore_support(
        all_hindi_tags,
        all_target_tags,
        average='macro',
        zero_division=0,
        labels=['PER', 'LOC', 'ORG']
    )

    # Exact match accuracy
    exact_matches = sum(1 for h, t in zip(all_hindi_tags, all_target_tags) if h == t)
    accuracy = exact_matches / len(all_hindi_tags) if all_hindi_tags else 0

    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'accuracy': accuracy,
        'entity_f1': entity_f1,
        'total_tokens': len(all_hindi_tags)
    }

# Evaluate all languages
print("="*70)
print(" METHOD 3: TOKEN-LEVEL ALIGNMENT ACCURACY")
print("="*70)
print("\nMeasures: How accurately were tags transferred at token level?\n")

all_results = {}

for filename, lang_name in language_files:
    print(f"\n{lang_name}:")
    print("-" * 50)

    # Load data
    with open(filename, 'r', encoding='utf-8') as f:
        data = [json.loads(line) for line in f]

    # Calculate metrics
    metrics = calculate_alignment_accuracy(data)

    print(f"  Accuracy:          {metrics['accuracy']:.3f} ({metrics['accuracy']*100:.1f}%)")
    print(f"  Precision:         {metrics['precision']:.3f}")
    print(f"  Recall:            {metrics['recall']:.3f}")
    print(f"  F1 Score:          {metrics['f1']:.3f}")
    print(f"  Entity F1:         {metrics['entity_f1']:.3f}")



    all_results[lang_name] = metrics

# Overall summary
print("\n" + "="*70)
print("📈 OVERALL SUMMARY - METHOD 3")
print("="*70)

avg_f1 = np.mean([r['f1'] for r in all_results.values()])
avg_entity_f1 = np.mean([r['entity_f1'] for r in all_results.values()])
avg_accuracy = np.mean([r['accuracy'] for r in all_results.values()])

print(f"\nAverage F1 Score:       {avg_f1:.3f}")
print(f"Average Entity F1:      {avg_entity_f1:.3f}")
print(f"Average Accuracy:       {avg_accuracy:.3f} ({avg_accuracy*100:.1f}%)")
